In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
x_train = x_train[:10000].astype(int) /255
y_train = y_train[:10000]

In [ ]:
def dp_side(array):
  initial = np.zeros(shape= array.shape)
  for i in range(len(initial[0])):
    if i == 0:
      initial[0][0] = array[0][0]
    else:
      initial[0][i] = array[0][i] + initial[0][i-1]
  
  for i in range(1, len(initial[:, 0])):
    initial[i][0] = array[i][0] + initial[i-1][0]

  for row in range(1, len(initial[1:,1]) + 1):
    for col in range(1, len(initial[1:, 1]) + 1):
      initial[row][col] = initial[row][col-1] + initial[row-1][col] - initial[row-1][col-1] + array[row][col]
  
  return initial


In [ ]:
new_arr = []
for i in x_train:
  k = dp_side(i)
  new_arr.append(k)
new_arr = np.array(new_arr)

In [ ]:
rects = []
while len(rects) != 100:
  top, bottom = (np.random.randint(0,28), np.random.randint(0,28)), (np.random.randint(0,28), np.random.randint(0,28))
  if abs(top[0] - bottom[0] >= 5) and abs(top[1] - bottom[1]) >= 5 and abs(top[0] - bottom[0]) * abs(top[1] - bottom[1]) >= 130:
    rects.append([top, bottom])

In [ ]:
def hor_diff(array):
  hor = []
  for i in rects:
    c1, c2 = i
    mid = abs(c1[0]-c2[0]) // 2
    if c1[0] < mid < c2[0]:
      if c1[1] <= c2[1]:
        k = (array[mid, c1[1]] - array[c1[0]-1, c1[1]] - array[mid, c2[1] -1] + array[c1[0]-1, c2[1]-1]) - (array[c2[0], c1[1]] - array[c2[0], c2[1] -1] - array[mid, c1[1]] + array[c2[0] -1, c2[1] -1])
      else:
        k = (array[mid, c2[1]] - array[c1[0]-1, c2[1]] - array[mid, c1[1] -1] + array[c1[0]-1, c1[1]-1]) - (array[c2[0], c2[1]] - array[c2[0], c1[1] -1] - array[mid, c2[1]] + array[c2[0] -1, c1[1] -1])
    
    if c1[0] > mid > c2[0]:
      if c1[1] <= c2[1]:
        k = (array[mid, c2[1]] - array[c2[0]-1, c2[1]] - array[mid, c1[1] -1] + array[c2[0]-1, c1[1]-1]) - (array[c1[0], c2[1]] - array[c1[0], c1[1] -1] - array[mid, c2[1]] + array[c1[0] -1, c1[1] -1])
      else:
        k = (array[mid, c1[1]] - array[c2[0]-1, c1[1]] - array[mid, c2[1] -1] + array[c2[0]-1, c2[1]-1]) - (array[c1[0], c1[1]] - array[c1[0], c2[1] -1] - array[mid, c1[1]] + array[c1[0] -1, c2[1] -1])
    hor.append(k)
  return hor

In [ ]:
def ver_diff(array):
  ver = []
  for i in rects:
    c1, c2 = i
    mid = abs(c1[1]-c2[1]) // 2
    if c1[1] < mid < c2[1]:
      if c1[0] <= c2[0]:
        k = (array[c2[0], mid] - array[c1[0]-1, mid] - array[c2[0] , c1[1] - 1] + array[c1[0]-1, c1[1]-1]) - (array[c2[0], c2[1]] - array[c2[0], mid] - array[c1[0] -1 , c2[1]] + array[c1[0]-1, mid])
      else:
        k = (array[c1[0], mid] - array[c1[0]-1, c1[1]] - array[c2[0] -1 , mid] + array[c1[0]-1, c1[1]-1]) - (array[c1[0], c2[1]] - array[c1[0], mid] - array[c2[0] -1 , c2[1]] + array[c2[0]-1, mid])
    
    if c1[1] > mid > c2[1]:
      if c1[1] <= c2[1]:
        k = (array[c1[0], mid] - array[c2[0]-1, mid] - array[c1[0] , c2[1] - 1] + array[c2[0]-1, c2[1]-1]) - (array[c1[0], c1[1]] - array[c1[0], mid] - array[c2[0] -1 , c1[1]] + array[c2[0]-1, mid])
      else:
        k = (array[c2[0], mid] - array[c2[0]-1, c2[1]] - array[c1[0] -1 , mid] + array[c2[0]-1, c2[1]-1]) - (array[c2[0], c1[1]] - array[c2[0], mid] - array[c1[0] -1 , c1[1]] + array[c1[0]-1, mid])
    ver.append(k)
  return ver

In [ ]:
diffs = []
for i in x_train:
  hor = hor_diff(i)
  ver = ver_diff(i)
  diffs.append(hor)
  diffs[-1].extend(ver)

In [ ]:
diffs = np.array(diffs)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [ ]:
x_tr, x_te, y_tr, y_te = train_test_split(diffs, y_train, test_size=0.2)

In [ ]:
len(y_tr)

8000

In [ ]:
log = LogisticRegression(max_iter=100000)
log.fit(x_tr, y_tr)

LogisticRegression(max_iter=100000)

In [ ]:
y_pred = log.predict(x_te)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_te, y_pred)

0.871